In [ ]:
import pandas as pd
from src import Utils, BERT

In [ ]:
MODEL_FOLDER = 'model_bert'

In [ ]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

In [ ]:
Utils.seed_random_number_generators()

In [ ]:
TEST_SIZE = 0.5

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
train_bert = BERT.BertPipeline

In [ ]:
# Batch size: 16, 32
# Learning rate (Adam): 5e-5, 3e-5, 2e-5
# Number of epochs: 2, 3, 4

train_bert.set_params(
  bert__batch_size=16,
  bert__optimizer__lr=5e-5,
  bert__max_epochs=2,
)

train_bert['bert'].callbacks[0].dirname = f'{MODEL_FOLDER}/train_bert'

In [ ]:
train_bert

In [ ]:
train_bert.fit(X_train, y_train)

In [ ]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_bert,
)

In [ ]:
Utils.save_trained_model(train_bert, f"{MODEL_FOLDER}/mBERT")

In [ ]:
history_data_frame = pd.DataFrame(
    train_bert['bert'].history
).set_index('epoch')
history_data_frame.to_csv(f'{MODEL_FOLDER}/bert_history.csv')

In [ ]:
history_data_frame

In [ ]:
metrics_data_frame = pd.DataFrame([{
  'accuracy': accuracy,
  'recall': recall,
  'precision': precision,
  'f1': f1,
}])
metrics_data_frame.to_csv(f'{MODEL_FOLDER}/bert_metrics.csv')